## SEP Image Detection & Basic Aperture Photometry Tutorial

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import sep
import astropy
from astropy.io import fits

rcParams['figure.figsize'] = [10., 8.]

### Read image into standard 2-d numpy array

In [ ]:
fname = "image.fits"
data = fits.getdata(fname)

### Show the image

In [ ]:
m = np.mean(data)
s = np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

#save this as the first image
plt.savefig('image1.png')

## Background Subtraction

### Measure a spatially varying background on the image

In [ ]:
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)

### Get a "global" mean and noise of the image background

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

### Evaluate background as 2-d array, same size as original image

In [ ]:
bkg_image = bkg.back()
#equivalent is bkg_image = np.array(bkg)

### Show the background

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()

#save this as the second image
plt.savefig('image2background.png')

### Evaluate the background noise as 2-d array, same size as original image

In [ ]:
bkg_rms = bkg.rms()

### Show the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin ='lower')
plt.colorbar()

#save this as the third image
plt.savefig('image3noise.png')

### Subtract the Background

In [ ]:
data_sub = data-bkg

## Object Detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

### How many objects were detected?

In [ ]:
print("There are",len(objects),"objects detected in this image.")

In [ ]:
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m = np.mean(data_sub)
s = np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i], angle=objects['theta'][i]* 180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

#save this as the final image
fig.savefig('image4detection.png')

### Available fields

In [ ]:
objects.dtype.names

## Aperture Photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)
#they are all 1-d arrays with one entry per object

### Show the first 10 objects results

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))